<a href="https://colab.research.google.com/github/kandamene/MU-TITAN/blob/main/Conflation_point_to_line.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conflation Code

In [ ]:
%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import urllib.request, json,os
from datetime import datetime

In [ ]:
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install --upgrade descartes
!pip install --upgrade rtree
!pip install fuzzywuzzy

In [ ]:
import geopandas as gpd
import shapely.geometry as geom
import rtree
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [ ]:
class conflation:
  """A simple class for conflating events to the roads they occured on"""
  def __init__(self,path,buffer):
    self.path = path
    self.buffer = buffer

  def get_events(self):
    with urllib.request.urlopen(self.path) as curl:
      json_data = json.loads(curl.read().decode())
    return json_data

  def url_to_data(self):
    event_data = self.get_events()
    df = pd.DataFrame()
    data_combo = []
    for cur_detector in event_data['data']:
      detector_data = cur_detector

      for col in detector_data['events']['event']:
        locations = col  
        cross_list = []
        for crosskys in locations['eventLocation']['crossStreets'].keys():
          cross_list.append(locations['eventLocation']['crossStreets'][crosskys])
        data_combo.append(cross_list)
    columns = ['atCrossStreet', 'latitude', 'longitude', 'onStreetName']
    df = pd.DataFrame(data_combo,columns=columns)
    test = []
    for i in df['onStreetName']:
      dir = i[-2:]
      if dir == 'NB':        test.append('N')
      elif dir == 'EB':        test.append('E')
      elif dir == 'SB':        test.append('S')
      elif dir == 'WB':        test.append('W')
      else:
        test.append('')
    df['DIRECTION'] = test
    return df

  def get_buffer(self):
    df = self.url_to_data()
    crashes = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    print('Using a buffer distance of: ', self.buffer)
    buffer_ = crashes.geometry.buffer(self.buffer)
    buffer_ = gpd.GeoDataFrame(buffer_)
    buffer_.columns =['geometry']
    buffer_.head()
    buffer_.crs = "EPSG:4326"
    roads_buffer = gpd.sjoin(roads, buffer_, how="inner", op='intersects')
    return roads_buffer

  def point_to_line(self):
    roads_buffer = self.get_buffer()
    df_ = self.url_to_data()
    test = roads_buffer.groupby(['index_right'])
    l = pd.DataFrame()
    for index, road_data in test:
      df__ = df_[df_.index==index]
      if road_data.shape[0] == 0:
        print('Buffer distance insufficient to locate possible roadway segments')
        break
      route = df__.iloc[0,3]  #find index of route names 
      routes = list(road_data['FULL_NAME'])
      a = road_data[road_data['FULL_NAME'] == process.extractOne(route,routes)[0]]
      l = pd.concat([l, a])
    df___ = pd.concat([df_.reset_index(drop=True), l.reset_index(drop=True)], axis=1)
    return df___

In [ ]:
# roads = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/MO Roads/MO_roads_and_weather/MO_roads_and_weather.shp")
# path = 'http://35.223.231.8:8080/api/transcore_incident/0'

In [ ]:
def run_conflation():  
  path = input('Enter your path to data url: ')
  buffer = float(input('Enter buffer: '))
  dp = conflation(path, buffer)
  df_data = dp.url_to_data()
  df_conflated = dp.point_to_line()
  return df_data, df_conflated


In [ ]:
# dp = conflation(path)
# df = dp.point_to_line()

# Run Conflation

In [ ]:
## Roads Shapefile Needs to loaded in first
## Save and import roads shapefiles from drive before proceeding 

roads = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/MO Roads/MO_roads_and_weather/MO_roads_and_weather.shp")

In [ ]:
## Run conflation by defining data url path and buffer distance
## Set minimum buffer distance to 0.0005

event_data, point_to_line = run_conflation()

Enter your path to data url: http://35.223.231.8:8080/api/transcore_incident/0
Enter buffer: 0.0005
Using a buffer distance of:  0.0005


In [ ]:
# Data of events parsed from url here

event_data

,atCrossStreet,latitude,longitude,onStreetName,DIRECTION
0,BEFORE I-270,38.553076,-90.420744,I-44 WB,W
1,ON RAMP FROM LINDBERGH BLVD,38.7795,-90.358904,I-270 WB,W
2,FORD LN CST,38.775836,-90.360016,PERSHALL RD CRD,
3,TO ELIZABETH AVE,38.772748,-90.309864,I-270 EB,E
4,BEFORE MO 367,38.770736,-90.251056,I-270 EB,E
...,...,...,...,...,...
247,AT KINGSHIGHWAY BLVD,38.61836,-90.26664,I-44 EB,E
248,BEFORE MO 100 EAST,38.483208,-90.832088,I-44 EB,E
249,PAST MASON RD,38.638484,-90.46872,I-64 EB,E
250,PAST I-64,38.645656,-90.448776,I-270 NB,N


In [ ]:
# Data of conflated events and combined roadway and closest weather station here

point_to_line

,atCrossStreet,latitude,longitude,onStreetName,DIRECTION,OBJECTID,TRAVELWAY_,NAME,DESG,OFFSET,DIRECTION,FULL_NAME,HIGHMEASUR,OPPOSING_T,Shape__Len,DIST_WCL,WCL_FID,long_WCL,lat_WCL,COUNTYNAME,COUNTYFIPS,COUNTYGNIS,NAME_UCASE,geometry,index_right
0,BEFORE I-270,38.553076,-90.420744,I-44 WB,W,365637,10,44,IS,None,W,IS 44 W,293.158,9,596754.484018,0.000190,470,-92.324807,37.779497,Pulaski,169,758539,PULASKI,"LINESTRING (-90.19036 38.64544, -90.18964 38.6...",0
1,ON RAMP FROM LINDBERGH BLVD,38.7795,-90.358904,I-270 WB,W,257059,6424,IS270W TO TAYLOR RD,RP,None,W,RP IS270W TO TAYLOR RD W,0.598,0,1236.173453,0.018957,300,-90.333831,38.782866,St Louis,189,758549,ST LOUIS,"LINESTRING (-90.35196 38.77734, -90.35634 38.7...",1
2,FORD LN CST,38.775836,-90.360016,PERSHALL RD CRD,,354491,223459,PERSHALL RD,CST,None,W,CST PERSHALL RD W,2.722,99568,5616.190126,0.008079,300,-90.333831,38.782866,St Louis,189,758549,ST LOUIS,"LINESTRING (-90.31719 38.77242, -90.31748 38.7...",2
3,TO ELIZABETH AVE,38.772748,-90.309864,I-270 EB,E,331165,6135,270,IS,None,E,IS 270 E,35.756,6136,73731.529213,0.000022,357,-90.448905,38.670452,St Louis,189,758549,ST LOUIS,"LINESTRING (-90.33365 38.50299, -90.33663 38.5...",3
4,BEFORE MO 367,38.770736,-90.251056,I-270 EB,E,331165,6135,270,IS,None,E,IS 270 E,35.756,6136,73731.529213,0.000022,357,-90.448905,38.670452,St Louis,189,758549,ST LOUIS,"LINESTRING (-90.33365 38.50299, -90.33663 38.5...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,AT KINGSHIGHWAY BLVD,38.61836,-90.26664,I-44 EB,E,365998,9,44,IS,None,E,IS 44 E,293.189,10,596820.790693,0.000371,470,-92.324807,37.779497,Pulaski,169,758539,PULASKI,"LINESTRING (-94.61787 36.99589, -94.61621 36.9...",247
248,BEFORE MO 100 EAST,38.483208,-90.832088,I-44 EB,E,365998,9,44,IS,None,E,IS 44 E,293.189,10,596820.790693,0.000371,470,-92.324807,37.779497,Pulaski,169,758539,PULASKI,"LINESTRING (-94.61787 36.99589, -94.61621 36.9...",248
249,PAST MASON RD,38.638484,-90.46872,I-64 EB,E,329646,6372,64,IS,None,E,IS 64 E,40.817,6373,84097.534407,0.001008,325,-90.819569,38.794816,St Charles,183,758546,ST CHARLES,"LINESTRING (-90.84214 38.80617, -90.84187 38.8...",249
250,PAST I-64,38.645656,-90.448776,I-270 NB,N,331165,6135,270,IS,None,E,IS 270 E,35.756,6136,73731.529213,0.000022,357,-90.448905,38.670452,St Louis,189,758549,ST LOUIS,"LINESTRING (-90.33365 38.50299, -90.33663 38.5...",250
